# Scraping Komentar YouTube Clash of Champions




Notebook ini berfungsi untuk mengumpulkan komentar-komentar dari video Clash of Champions yang diselenggarakan oleh Ruangguru dan dapat ditonton melalui youtube.

## Impor Library

In [1]:
import pandas as pd
from googleapiclient.discovery import build

## Data Gathering

Sebagai gambaran, Clash of Champions ditayangkan dalam berbagai episode. Hingga Notebook ini dibuat, yaitu 5 Agustus 2024 pukul 16.45 WIB, ada 9 episode yang telah ditayangkan di YouTube. Notebook ini akan mengumpulkan komentar-komentar dari 9 episode tersebut.

In [2]:
# Mendefinisikan API key
API_KEY = "YOUR API KEY"
YOUTUBE_API_SERVICE_NAME = "youtube"
YOUTUBE_API_VERSION = "v3"

# Membuat fungsi untuk mengumpulkan komen yang ada di youtube
def get_video_comments(video_id):
    youtube = build(YOUTUBE_API_SERVICE_NAME, YOUTUBE_API_VERSION,
                    developerKey = API_KEY)
    comments = []

    # Mengoleksi komentar
    request = youtube.commentThreads().list(
        part = "snippet",
        videoId = video_id,
        maxResults = 100
    )

    # Mengoleksi komentar serta pengunggah komentar
    while request:
        response = request.execute()
        for item in response["items"]:
            comment = item["snippet"]["topLevelComment"]["snippet"]
            comments.append({
                "author": comment["authorDisplayName"],
                "comment": comment["textDisplay"]
            })
        request = youtube.commentThreads().list_next(request, response)
    return comments

# Membuat fungsi untuk menyimpan komentar ke dalam file csv
def save_comments_to_csv(comments, filename):
    data = pd.DataFrame(comments)
    data.to_csv(filename, index = False)

if __name__ == "__main__":
    # Membuat list id video yang ingin dikumpulkan komentarnya
    video_ids = ["dY_3acmFFRQ", "n2sggHMSAro", "IFG-F5HJDpg",
                 "_2yf29Wdekc", "lDAIIkm3T_o", "1jHk08eavXU",
                 "H-TPKUdRGEY", "qlzIkTCsJSM", "4Ls4-oRBtS4"]
    all_comments = []

    # Melakukan iterasi untuk mendapat komentar dari setiap id video
    for i, video_id in enumerate(video_ids):
        comments = get_video_comments(video_id)
        all_comments.extend(comments)

    # Mengombinasikan seluruh komentar menjadi satu file di seluruh_komen.csv
    save_comments_to_csv(all_comments, "seluruh_komen.csv")
    print("Berhasil! Seluruh komentar telah berhasil tersimpan.")

Berhasil! Seluruh komentar telah berhasil tersimpan.


In [3]:
# Menampilkan 10 baris pertama dari komentar
data = pd.read_csv("seluruh_komen.csv")
data.head(10)

,author,comment
0,@Ruangguru,"Yuk, konsultasi dan klaim diskonnya sekarang d..."
1,@maghda.n,keren
2,@arisandi6883,Syakiraaaa ILOVEU
3,@stevendalmat,Aku suka film ini
4,@fikaazura1624,Tes pauli atau tes koran dalam tes psikotes ku...
5,@askiazalianti897,Siapa yg kesini karna rewatch☝️
6,@dickyfirmansyah6729,Keren bgt sih ❤
7,@tatokki0927,AAAAHHH SERU BANGETT
8,@nurjannah7090,"Masyaa Alloh Sehat,Semangat,Berkah,Bahagia,Kom..."
9,@ZahraSyafira-hg1il,"saran aja ya, kalo bisa hostnya ngomong full b..."


## Data Assessing

In [4]:
# Menampilkan ukuran data
data.shape

(63805, 2)

Perlu diperhatikan bahwa jika seluruh kode dijalankan ulang, sangat mungkin ukuran data akan berubah-ubah karena bisa saja komentar yang ada di YouTube selalu bertambah.

In [5]:
# Menampilkan informasi umum terkait data
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 63805 entries, 0 to 63804
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   author   63805 non-null  object
 1   comment  63805 non-null  object
dtypes: object(2)
memory usage: 997.1+ KB


Dari hasil info di atas, terlihat bahwa kedua kolom memiliki 63805 baris yang non-null. Artinya, tidak ada data bernilai null, sehingga cukup diperiksa apakah terdapat duplikasi data.

In [6]:
# Menghitung jumlah data duplikat
data.duplicated().sum()

790

Terlihat bahwa terdapat 790 data duplikat. Akan diperiksa data-data duplikat tersebut apakah memang duplikat atau tidak.

In [7]:
# Menampilkan data duplikat
data[data.duplicated]

,author,comment
136,@nurdinsans384,Hanif kertasnya kosong gak ada coretan 🗿pertam...
247,@rendlessism,Ngeri ya ITB
512,@SamsungUtama-r6k,Ya allah yg ke tiga ngeliatnya aja udh pusing😭
573,@tahsigungreti8816,"Sambil nangis dari awal nonton, karena mikir k..."
574,@tahsigungreti8816,"Sambil nangis dari awal nonton, karena mikir k..."
...,...,...
63288,@ashabella3460,XAVIERA❤❤❤❤❤❤
63446,@indahsasoe,nangis
63492,@nayyacantik-wl1xz,Aaaaaa sendy ❤❤❤❤❤
63548,@olgaolifiani7951,"Sebagai warga Indonesia, saya benar benar bang..."


Karena terdapat data duplikat, maka data duplikat akan dihapus.

## Data Cleaning

In [8]:
# Menghapus data duplikat, lalu menampilkan informasi data setelah data duplikat dihapus
data.drop_duplicates(inplace = True)
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 63015 entries, 0 to 63804
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   author   63015 non-null  object
 1   comment  63015 non-null  object
dtypes: object(2)
memory usage: 1.4+ MB


In [9]:
# Menampilkan 10 baris pertama dari komentar setelah data duplikat dihapus
data.head(10)

,author,comment
0,@Ruangguru,"Yuk, konsultasi dan klaim diskonnya sekarang d..."
1,@maghda.n,keren
2,@arisandi6883,Syakiraaaa ILOVEU
3,@stevendalmat,Aku suka film ini
4,@fikaazura1624,Tes pauli atau tes koran dalam tes psikotes ku...
5,@askiazalianti897,Siapa yg kesini karna rewatch☝️
6,@dickyfirmansyah6729,Keren bgt sih ❤
7,@tatokki0927,AAAAHHH SERU BANGETT
8,@nurjannah7090,"Masyaa Alloh Sehat,Semangat,Berkah,Bahagia,Kom..."
9,@ZahraSyafira-hg1il,"saran aja ya, kalo bisa hostnya ngomong full b..."


Data sudah siap untuk diproses agar bisa dilatih dengan Machine Learning.